In [1]:

# Week 3 - Section 2: MLflow Experiment Tracking (Business)
# Single Code Cell Execution — with Profiles: dev, preprod, final
# Includes: Upsert into consolidated Week 3 report (no duplicates).

import os, re, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

warnings.filterwarnings("ignore")


import re

def upsert_section_in_consolidated(consolidated_path: Path, section_text: str,
                                   header_variants: list[str],
                                   insert_order_hint: list[str] | None = None):
    """
    Replace the block that starts with any header in `header_variants`.
    If none exists, append (or insert before the first hint header, if provided).
    Robust to '-' vs '–' and avoids inline regex flags collisions.
    """
    consolidated_path.parent.mkdir(parents=True, exist_ok=True)
    existing = consolidated_path.read_text(encoding="utf-8") if consolidated_path.exists() else ""
    text = existing.replace("\r\n", "\n").replace("\r", "\n")

    # Build regex to remove any existing block for this section
    hdr_alt = "|".join(re.escape(h) for h in header_variants)
    any_sds_hdr = r"^\#\s*SDS-CP036-powercast\s*[–-]\s*Week\s*3\s*Section\s*\d+:\s*.*$"
    block_pat = rf"^(?:{hdr_alt})\s*.*?(?=^{any_sds_hdr}|\Z)"
    text = re.sub(block_pat, "", text, flags=re.M | re.S).strip()

    # Prepare the new (clean) block to insert
    new_block = section_text.strip()

    def insert_before_first_hint(container: str, block: str, hints: list[str]) -> str:
        for h in hints:
            m = re.search(rf"^{re.escape(h)}\s*$", container, flags=re.M)
            if m:
                return container[:m.start()] + (block + "\n\n---\n\n") + container[m.start():]
        return container + ("\n\n---\n\n" if container.strip() else "") + block

    if insert_order_hint:
        text = insert_before_first_hint(text, new_block, insert_order_hint)
    else:
        text = text + ("\n\n---\n\n" if text.strip() else "") + new_block

    consolidated_path.write_text(text.strip() + "\n", encoding="utf-8")


# ---------- Optional model deps (graceful fallback) ----------
XGB_AVAILABLE = True
try:
    from xgboost import XGBRegressor
except Exception:
    XGB_AVAILABLE = False

SARIMAX_AVAILABLE = True
try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
except Exception:
    SARIMAX_AVAILABLE = False

MLFLOW_AVAILABLE = True
try:
    import mlflow
    import mlflow.sklearn
except Exception:
    MLFLOW_AVAILABLE = False

# ---------- Project paths & helpers ----------
BASE_PROJECT_NAME = "SDS-CP036-powercast"

def find_repo_root(start: Path) -> Path:
    cur = start
    for _ in range(12):
        if (cur / "data").exists():
            return cur
        cur = cur.parent
    return start  # fallback

BASE_DIR = find_repo_root(Path.cwd())

# ---------- Profiles (dev, preprod, final) ----------
PROFILE = "dev"  # choose: "dev", "preprod", "final"

profiles = {
    "dev":     dict(FAST_MODE=True,  RESAMPLE_TO="H", MAX_DAYS=365,
                    TEST_DAYS=7,  BACKTEST=False, BACKTEST_FOLDS=0, BACKTEST_STEP_DAYS=0, BACKTEST_HOURS=0),
    "preprod": dict(FAST_MODE=False, RESAMPLE_TO="H", MAX_DAYS=365,
                    TEST_DAYS=28, BACKTEST=False, BACKTEST_FOLDS=0, BACKTEST_STEP_DAYS=0, BACKTEST_HOURS=0),
    "final":   dict(FAST_MODE=False, RESAMPLE_TO="H", MAX_DAYS=365,
                    TEST_DAYS=None, BACKTEST=True, BACKTEST_FOLDS=6, BACKTEST_STEP_DAYS=7, BACKTEST_HOURS=168),
}
cfg = profiles[PROFILE]

FAST_MODE   = cfg["FAST_MODE"]
RESAMPLE_TO = cfg["RESAMPLE_TO"]
MAX_DAYS    = cfg["MAX_DAYS"]
TEST_DAYS   = cfg["TEST_DAYS"]
BACKTEST    = cfg["BACKTEST"]
BACKTEST_FOLDS = cfg["BACKTEST_FOLDS"]
BACKTEST_STEP_DAYS = cfg["BACKTEST_STEP_DAYS"]
BACKTEST_HOURS = cfg["BACKTEST_HOURS"]

# Results per profile (keeps outputs separate)
RESULTS_DIR = BASE_DIR / "results" / f"Wk03_Section2_{PROFILE}"
PLOTS_DIR   = RESULTS_DIR / "plots"
REPORTS_DIR = RESULTS_DIR / "reports"
for d in [RESULTS_DIR, PLOTS_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ---------- MLflow config ----------
EXPERIMENT_NAME = "powercast_wk03_s2"
LOCAL_MLRUNS = BASE_DIR / "mlruns"  # file-based tracking inside repo

if MLFLOW_AVAILABLE:
    try:
        mlflow.set_tracking_uri(f"file://{LOCAL_MLRUNS.as_posix()}")
        mlflow.set_experiment(EXPERIMENT_NAME)
    except Exception as e:
        print("[WARN] MLflow setup skipped:", e)
        MLFLOW_AVAILABLE = False

# ---------- Load & prepare data ----------
data_path = BASE_DIR / "data" / "Tetuan City power consumption.csv"
if not data_path.exists():
    raise FileNotFoundError(f"Dataset not found at: {data_path}")

df = pd.read_csv(data_path)
# Normalize column names (strip & collapse spaces like 'Zone 2  Power ...')
df.columns = df.columns.str.strip().str.replace(r"\s+", " ", regex=True)

# Parse time & set index
df["DateTime"] = pd.to_datetime(df["DateTime"])
df = df.set_index("DateTime").sort_index()

# Keep numeric cols only for modeling safety
num_df = df.select_dtypes(include=[np.number]).copy()

# Downsample & cap horizon
if RESAMPLE_TO:
    num_df = num_df.resample(RESAMPLE_TO).mean()
if isinstance(MAX_DAYS, (int, float)):
    try:
        num_df = num_df.last(f"{int(MAX_DAYS)}D")
    except Exception:
        num_df = num_df.iloc[-24*int(MAX_DAYS):]

zones = ["Zone 1 Power Consumption", "Zone 2 Power Consumption", "Zone 3 Power Consumption"]
zones = [z for z in zones if z in num_df.columns]

# ---------- Helpers ----------
def mape_safe(y_true, y_pred):
    denom = np.where(y_true == 0, np.nan, np.abs(y_true))
    return float(np.nanmean(np.abs(y_true - y_pred) / denom) * 100.0)

def evaluate_forecast(y_true, y_pred):
    return {
        "RMSE": float(mean_squared_error(y_true, y_pred, squared=False)),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "MAPE": mape_safe(np.asarray(y_true), np.asarray(y_pred)),
    }

def plot_and_save(zone, y_true_index, y_true, y_pred, model_name):
    plt.figure(figsize=(11,4))
    plt.plot(y_true_index, y_true, label="Actual")
    plt.plot(y_true_index, y_pred, label=f"Predicted ({model_name})")
    plt.title(f"{zone} - {model_name}")
    plt.legend()
    safe_zone = re.sub(r"[^A-Za-z0-9_.-]+","_", zone)
    safe_model = re.sub(r"[^A-Za-z0-9_.-]+","_", model_name)
    fname = PLOTS_DIR / f"{safe_zone}_{safe_model}.png"
    plt.savefig(fname, bbox_inches="tight")
    plt.close()
    return fname

def log_run_mlflow(zone, model_name, params, metrics, artifacts_paths):
    if not MLFLOW_AVAILABLE:
        return None
    with mlflow.start_run(run_name=f"{PROFILE}__{zone}__{model_name}"):
        mlflow.log_params({
            "profile": PROFILE,
            "zone": zone,
            "resample": RESAMPLE_TO,
            "max_days": MAX_DAYS,
            "fast_mode": FAST_MODE,
            **params,
        })
        mlflow.log_metrics(metrics)
        for p in artifacts_paths:
            try:
                mlflow.log_artifact(str(p))
            except Exception as e:
                print("[WARN] Could not log artifact", p, ":", e)
        mlflow.set_tag("section", "Wk03_Section2")
        return mlflow.active_run().info.run_id

def rolling_splits(index, folds=4, step_days=7, horizon_hours=168):
    step = pd.Timedelta(days=step_days)
    horizon = pd.Timedelta(hours=horizon_hours)
    end = index.max()
    splits = []
    cur_test_end = end
    for fold in range(folds):
        test_start = cur_test_end - horizon + pd.Timedelta(hours=1)
        val_end = test_start - pd.Timedelta(hours=1)
        val_start = val_end - step + pd.Timedelta(hours=1)
        train_end = val_start - pd.Timedelta(hours=1)
        if train_end <= index.min():
            break
        tr_mask = (index <= train_end)
        va_mask = (index >= val_start) & (index <= val_end)
        te_mask = (index >= test_start) & (index <= cur_test_end)
        splits.append((tr_mask, va_mask, te_mask))
        cur_test_end = test_start - pd.Timedelta(hours=1)
    return splits

# ---------- Experiment execution ----------
summary_rows = []

try:
    if not BACKTEST:
        # Single hold-out mode
        if TEST_DAYS is None:
            TEST_DAYS = 7
        test = num_df.last(f"{TEST_DAYS}D")
        pre = num_df.iloc[: -len(test)] if len(num_df) > len(test) else num_df.iloc[:0]
        n_pre = len(pre)
        n_train = int(n_pre * 0.85)  # 85% train, 15% val
        train = pre.iloc[:n_train]
        val = pre.iloc[n_train:]

        # Baseline (Naive)
        for zone in zones:
            if len(val) == 0 or len(test) == 0:
                continue
            y_true = test[zone].values
            pivot = val[zone].iloc[-1] if len(val) else train[zone].iloc[-1]
            y_pred = np.repeat(pivot, len(test))
            metrics = evaluate_forecast(y_true, y_pred)
            plot_file = plot_and_save(zone, test.index, y_true, y_pred, "Baseline (Naive)")
            run_id = log_run_mlflow(zone, "Baseline (Naive)", {"model": "baseline"}, metrics, [plot_file])
            summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "Baseline (Naive)", **metrics, "Fold": None, "mlflow_run_id": run_id})

        # SARIMAX (light)
        if SARIMAX_AVAILABLE:
            for zone in zones:
                try:
                    if len(train) < 10 or len(test) == 0:
                        continue
                    model = SARIMAX(train[zone], order=(1,1,1), seasonal_order=(0,1,1,24),
                                    enforce_stationarity=False, enforce_invertibility=False)
                    res = model.fit(disp=False)
                    fc = res.get_forecast(steps=len(test)).predicted_mean
                    y_pred = fc.values
                    y_true = test[zone].values
                    metrics = evaluate_forecast(y_true, y_pred)
                    plot_file = plot_and_save(zone, test.index, y_true, y_pred, "SARIMAX(1,1,1)(0,1,1,24)")
                    run_id = log_run_mlflow(zone, "SARIMAX(1,1,1)(0,1,1,24)",
                                            {"model": "sarimax", "order": "(1,1,1)", "seasonal_order": "(0,1,1,24)"},
                                            metrics, [plot_file])
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "SARIMAX(1,1,1)(0,1,1,24)", **metrics, "Fold": None, "mlflow_run_id": run_id})
                except Exception as e:
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "SARIMAX", "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "Fold": None, "mlflow_run_id": None})

        # XGBoost with lag features (if available)
        if XGB_AVAILABLE:
            def make_lag_df(series, lags=24):
                df_l = pd.DataFrame({"y": series})
                for L in range(1, lags+1):
                    df_l[f"lag_{L}"] = df_l["y"].shift(L)
                return df_l.dropna()

            LAGS = 24
            for zone in zones:
                try:
                    series = pd.concat([train[zone], val[zone], test[zone]])
                    df_lag = make_lag_df(series, lags=LAGS)
                    n_train_lag = len(train)
                    n_val_lag = len(val)
                    split_idx = n_train_lag + n_val_lag - LAGS
                    train_ml = df_lag.iloc[:split_idx]
                    test_ml = df_lag.iloc[split_idx:]
                    X_tr, y_tr = train_ml.drop(columns=["y"]), train_ml["y"]
                    X_te, y_te = test_ml.drop(columns=["y"]), test_ml["y"]
                    model = XGBRegressor(
                        n_estimators=120 if FAST_MODE else 200,
                        max_depth=4 if FAST_MODE else 6,
                        learning_rate=0.1,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        objective="reg:squarederror",
                        n_jobs=0
                    )
                    model.fit(X_tr, y_tr, verbose=False)
                    y_pred = model.predict(X_te)
                    y_true_idx = test.index[-len(y_pred):]
                    y_true = test[zone].reindex(y_true_idx).values
                    metrics = evaluate_forecast(y_true, y_pred)
                    plot_file = plot_and_save(zone, y_true_idx, y_true, y_pred, "XGBoost (lags)")
                    run_id = log_run_mlflow(zone, "XGBoost (lags)",
                                            {"model": "xgboost_lags", "lags": LAGS, "n_estimators": int(model.get_params().get("n_estimators", 0))},
                                            metrics, [plot_file])
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "XGBoost (lags)", **metrics, "Fold": None, "mlflow_run_id": run_id})
                except Exception as e:
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "XGBoost (lags)", "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "Fold": None, "mlflow_run_id": None})

        results_df = pd.DataFrame(summary_rows)
        results_csv = REPORTS_DIR / "mlflow_run_summary.csv"
        results_df.to_csv(results_csv, index=False)

    else:
        # Rolling backtest mode
        idx = num_df.index
        splits = rolling_splits(idx, folds=BACKTEST_FOLDS, step_days=BACKTEST_STEP_DAYS, horizon_hours=BACKTEST_HOURS)

        for zone in zones:
            for fold_id, (tr_m, va_m, te_m) in enumerate(splits, start=1):
                train = num_df.loc[tr_m]
                val   = num_df.loc[va_m]
                test  = num_df.loc[te_m]

                # Baseline
                try:
                    y_true = test[zone].values
                    pivot = val[zone].iloc[-1] if len(val) else train[zone].iloc[-1]
                    y_pred = np.repeat(pivot, len(test))
                    metrics = evaluate_forecast(y_true, y_pred)
                    run_id = log_run_mlflow(zone, "Baseline (Naive)",
                                            {"model": "baseline", "fold": fold_id, "horizon_h": BACKTEST_HOURS},
                                            metrics, [])
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "Baseline (Naive)", **metrics, "Fold": fold_id, "mlflow_run_id": run_id})
                except Exception:
                    summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "Baseline (Naive)", "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "Fold": fold_id, "mlflow_run_id": None})

                # SARIMAX
                if SARIMAX_AVAILABLE:
                    try:
                        if len(train) > 10:
                            model = SARIMAX(train[zone], order=(1,1,1),
                                            seasonal_order=(0,1,1,24),
                                            enforce_stationarity=False, enforce_invertibility=False)
                            res = model.fit(disp=False)
                            y_pred = res.get_forecast(steps=len(test)).predicted_mean.values
                            metrics = evaluate_forecast(y_true, y_pred)
                            run_id = log_run_mlflow(zone, "SARIMAX(1,1,1)(0,1,1,24)",
                                                    {"model": "sarimax", "order": "(1,1,1)", "seasonal_order": "(0,1,1,24)",
                                                     "fold": fold_id, "horizon_h": BACKTEST_HOURS},
                                                    metrics, [])
                            summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "SARIMAX(1,1,1)(0,1,1,24)", **metrics, "Fold": fold_id, "mlflow_run_id": run_id})
                    except Exception:
                        summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "SARIMAX", "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "Fold": fold_id, "mlflow_run_id": None})

                # XGBoost (lags)
                if XGB_AVAILABLE:
                    try:
                        def make_lag_df(series, lags=24):
                            df_l = pd.DataFrame({"y": series})
                            for L in range(1, lags+1):
                                df_l[f"lag_{L}"] = df_l["y"].shift(L)
                            return df_l.dropna()

                        LAGS = 24
                        series = pd.concat([train[zone], val[zone], test[zone]])
                        df_lag = make_lag_df(series, lags=LAGS)
                        n_train_lag = len(train)
                        n_val_lag = len(val)
                        split_idx = n_train_lag + n_val_lag - LAGS
                        train_ml = df_lag.iloc[:split_idx]
                        test_ml = df_lag.iloc[split_idx:]
                        X_tr, y_tr = train_ml.drop(columns=["y"]), train_ml["y"]
                        X_te, y_te = test_ml.drop(columns=["y"]), test_ml["y"]

                        model = XGBRegressor(
                            n_estimators=150,
                            max_depth=5,
                            learning_rate=0.08 if not FAST_MODE else 0.1,
                            subsample=0.9,
                            colsample_bytree=0.9,
                            objective="reg:squarederror",
                            n_jobs=0
                        )
                        model.fit(X_tr, y_tr, verbose=False)
                        y_pred = model.predict(X_te)
                        y_true_idx = test.index[-len(y_pred):]
                        y_true = test[zone].reindex(y_true_idx).values
                        metrics = evaluate_forecast(y_true, y_pred)
                        run_id = log_run_mlflow(zone, "XGBoost (lags)",
                                                {"model": "xgboost_lags", "lags": LAGS, "n_estimators": int(model.get_params().get("n_estimators", 0)),
                                                 "fold": fold_id, "horizon_h": BACKTEST_HOURS},
                                                metrics, [])
                        summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "XGBoost (lags)", **metrics, "Fold": fold_id, "mlflow_run_id": run_id})
                    except Exception:
                        summary_rows.append({"Profile": PROFILE, "Zone": zone, "Model": "XGBoost (lags)", "RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "Fold": fold_id, "mlflow_run_id": None})

        # Save fold-level results + aggregates
        results_df = pd.DataFrame(summary_rows)
        results_csv = REPORTS_DIR / "mlflow_run_summary_folds.csv"
        results_df.to_csv(results_csv, index=False)

        agg = results_df.groupby(["Zone","Model"]).agg(
            RMSE_mean=("RMSE","mean"), RMSE_std=("RMSE","std"),
            MAE_mean=("MAE","mean"),   MAE_std=("MAE","std"),
            MAPE_mean=("MAPE","mean"), MAPE_std=("MAPE","std")
        ).reset_index()
        agg_csv = REPORTS_DIR / "mlflow_aggregate_summary.csv"
        agg.to_csv(agg_csv, index=False)

        champs = agg.sort_values(["Zone","RMSE_mean"]).groupby("Zone").head(1)
        champs_csv = REPORTS_DIR / "mlflow_champions.csv"
        champs.to_csv(champs_csv, index=False)

finally:
    # ---------- Build the Section Report (Core + ALWAYS Business Summary) ----------
    section_report_path = REPORTS_DIR / "SDS-CP036-powercast_Wk03_Section2_Business_Report.md"
    consolidated_report_path = BASE_DIR / "SDS-CP036-powercast_Wk03_Report_Business.md"

    if not BACKTEST:
        csv_link_line = "[Run Summary - CSV](mlflow_run_summary.csv)"
        extra_lines = []
    else:
        csv_link_line = "[Run Summary (folds) - CSV](mlflow_run_summary_folds.csv)"
        extra_lines = [
            "[Aggregate Summary - CSV](mlflow_aggregate_summary.csv)",
            "[Champion Models - CSV](mlflow_champions.csv)",
        ]

    core_md = [
        f"# {BASE_PROJECT_NAME} - Week 3 Section 2: MLflow Experiment Tracking",
        "",
        f"Profile: **{PROFILE}**",
        "",
        "## Key Questions Answered",
        "",
        "Q: Which evaluation metrics did you use to assess model performance, and why are they appropriate for this problem?",
        "A: We used RMSE, MAE, and MAPE. RMSE penalizes large errors (useful for risk), MAE provides an easy-to-explain average miss, and MAPE gives a percentage error that business users understand.",
        "",
        "Q: How did you use MLflow to track your experiments and results?",
        f"A: We created an MLflow experiment named {EXPERIMENT_NAME} and logged, for each run:",
        "- Parameters: model name, resampling (hour/hourly), lags, seasonal order, profile tag, fast_mode flag.",
        "- Metrics: RMSE, MAE, MAPE for the evaluation window.",
        "- Artifacts: Actual vs Predicted plots per zone (in single hold-out), or per fold (in backtesting).",
        "Tracking runs locally via a file-based MLflow backend keeps everything versioned inside the repo (mlruns/).",
        "",
        "Q: What insights did you gain from comparing actual vs. predicted curves for each zone?",
        "A: Visual comparisons showed how models capture shape (daily patterns), timing (peaks/valleys), and magnitude (over/under-bias). These help pinpoint which model is most reliable for each zone's operations.",
        "",
        csv_link_line,
    ] + ([""] + extra_lines if extra_lines else [])

    business_md_always = [
        "---",
        "",
        "## Business Value Summary (Executive View)",
        "- Transparency & Trust: Every experiment is versioned with parameters, metrics, and plots so leaders can see how conclusions were reached.",
        '- Faster Decisions: Side-by-side comparisons shorten the path to a "champion" model for each zone.',
        "- Risk Control: Using RMSE/MAE/MAPE together prevents over-optimizing for a single metric and missing real-world errors.",
        "- Governance & Auditability: A local MLflow history (mlruns/) provides an auditable trail for compliance, board reviews, or customer assurances.",
        "- Scalability: Profiles (dev/preprod/final) let us move from quick smoke-tests to robust selection without changing code.",
    ]

    REPORTS_DIR.mkdir(parents=True, exist_ok=True)
    with open(section_report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(core_md + [""] + business_md_always))

    # ===== Upsert into consolidated (replace-or-append) =====
    SECTION1_HEADERS = [
        "# SDS-CP036-powercast - Week 3 Section 1: Model Selection & Training",
        "# SDS-CP036-powercast – Week 3 Section 1: Model Selection & Training",
    ]
    SECTION2_HEADERS = [
        "# SDS-CP036-powercast - Week 3 Section 2: MLflow Experiment Tracking",
        "# SDS-CP036-powercast – Week 3 Section 2: MLflow Experiment Tracking",
    ]
    section_text = Path(section_report_path).read_text(encoding="utf-8")
    upsert_section_in_consolidated(
        consolidated_path=consolidated_report_path,
        section_text=section_text,
        header_variants=SECTION2_HEADERS,
        insert_order_hint=SECTION1_HEADERS  # prefer after Section 1 if present
    )

    # Helpful debug info
    print("Profile:", PROFILE)
    print("MLflow available:", MLFLOW_AVAILABLE)
    print("Results dir:", RESULTS_DIR.resolve())
    print("Section report path:", section_report_path.resolve())
    print("Consolidated report path:", consolidated_report_path.resolve())
    print("Done upserting Section 2.")


Profile: dev
MLflow available: False
Results dir: /home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk03_Section2_dev
Section report path: /home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk03_Section2_dev/reports/SDS-CP036-powercast_Wk03_Section2_Business_Report.md
Consolidated report path: /home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/SDS-CP036-powercast_Wk03_Report_Business.md
Done upserting Section 2.
